In [1]:
import pandas as pd
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy.stats import chi2
import matplotlib.dates as mdates
import random
from sklearn.model_selection import train_test_split

In [6]:
df_l8 = np.array([
        [1,1,1,1,1,1,1],
        [1,1,1,2,2,2,2],
        [1,2,2,1,1,2,2],
        [1,2,2,2,2,1,1],
        [2,1,2,1,2,1,2],
        [2,1,2,2,1,2,1],
        [2,2,1,1,2,2,1],
        [2,2,1,2,1,1,2]
        ])
df_l8==1

array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False, False, False, False],
       [ True, False, False,  True,  True, False, False],
       [ True, False, False, False, False,  True,  True],
       [False,  True, False,  True, False,  True, False],
       [False,  True, False, False,  True, False,  True],
       [False, False,  True,  True, False, False,  True],
       [False, False,  True, False,  True,  True, False]])

In [11]:
np.zeros(12)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [8]:
df_l8 = pd.DataFrame([
        [1,1,1,1,1,1,1],
        [1,1,1,2,2,2,2],
        [1,2,2,1,1,2,2],
        [1,2,2,2,2,1,1],
        [2,1,2,1,2,1,2],
        [2,1,2,2,1,2,1],
        [2,2,1,1,2,2,1],
        [2,2,1,2,1,1,2]
        ])
(df_l8==1).values

array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False, False, False, False],
       [ True, False, False,  True,  True, False, False],
       [ True, False, False, False, False,  True,  True],
       [False,  True, False,  True, False,  True, False],
       [False,  True, False, False,  True, False,  True],
       [False, False,  True,  True, False, False,  True],
       [False, False,  True, False,  True,  True, False]])

## データの取得

In [91]:
df = pd.read_csv('../data/letter_recognition.csv', header=None)

#Aのみを判定するため，Aを１，A以外を0にした．
df[0] = df[0].apply(lambda x: 1 if x == 'A' else 0)

#Xとyを入力
X = df[range(1,17)]
y = df[0]



In [92]:
#バギング側の話
#ブートストラップサンプリングの個数
n = 10
seed = random.randint(0, n)

#使用する7つの変数をランダムに取得する
random.seed(1)
random_s = random.sample(list(X.columns), 7)
use_X = X[random_s]

X_train, X_test, y_train, y_test = train_test_split(use_X, y, test_size=0.2)


### Xとyが入力されたら自動的にMDを求めるためのscalerとinv_covと使用する変数(select_columns)を出力するアルゴリズムを作る

In [93]:
normal_idx = y_train[y_train == 0].index.to_list()
anomaly_idx = y_train[y_train == 1].index.to_list()

#通常状態の説明変数と目的変数を定義
normal_X = X_train.loc[normal_idx]
normal_y = y_train.loc[normal_idx]
anomaly_X = X_train.loc[anomaly_idx]
anomaly_y = y_train.loc[anomaly_idx]

In [94]:
normal_X.to_csv('../data/normal_X.csv')

## 必要な関数の定義

In [95]:
#各説明変数ごとに標準化
def transform_standard(fit_X, transform_X):
    scaler = StandardScaler()
    scaler.fit(fit_X)
    return scaler.transform(transform_X)

#共分散行列の逆行列
def inv_cov(x):
    #標準化後のベクトルを入力する
    #標準化した後なので相関行列と分散共分散行列は一致する
    cov = np.cov(x.T)
    return np.linalg.inv(cov)

#マハラノビス汎距離
def cal_MD(Z, inv_C, k):
    MD = []
    for i in range(len(Z)):
        _a = np.dot(Z[i], inv_C)
        _MD = np.dot(_a, Z[i].T)
        _MD = _MD / k
        MD.append(_MD)
    return MD


In [96]:
#標準化
normal_Z = transform_standard(normal_X, normal_X)
anomaly_Z = transform_standard(normal_X, anomaly_X) #異常データも正常データの平均と分散で標準化

#共分散行列の逆行列
inv_C = inv_cov(normal_Z)

#マハラノビス距離の計算
normal_MD = cal_MD(normal_Z, inv_C, len(use_X.columns))
anomaly_MD = cal_MD(anomaly_Z, inv_C, len(use_X.columns))


## ここまではあってる

### 正常データの中の異常なデータを任意に削除することもある

In [97]:
# def del_anomaly(x):
#     #マハラノビス距離がx以上の通常データを削除する
#     df_MD = pd.DataFrame(index=normal_idx, data=normal_MD)
#     del_anomaly_idx = df_MD.loc[df_MD[0] > x].index.to_list()
#     for i in del_anomaly_idx:
#         normal_idx.remove(i)
#     return normal_idx
# len(del_anomaly(3))

In [98]:
#L8直行表
df_l8 = pd.DataFrame([[1,1,1,1,1,1,1],[1,1,1,2,2,2,2],[1,2,2,1,1,2,2],[1,2,2,2,2,1,1],[2,1,2,1,2,1,2],[2,1,2,2,1,2,1],[2,2,1,1,2,2,1],[2,2,1,2,1,1,2]])
l8 = (df_l8==1).values
l8

array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False, False, False, False],
       [ True, False, False,  True,  True, False, False],
       [ True, False, False, False, False,  True,  True],
       [False,  True, False,  True, False,  True, False],
       [False,  True, False, False,  True, False,  True],
       [False, False,  True,  True, False, False,  True],
       [False, False,  True, False,  True,  True, False]])

In [99]:
#異常データのマハラノビス距離
result = np.zeros((l8.shape[0], anomaly_Z.shape[0]))
for x, l8_row in enumerate(l8):
    result[x] = cal_MD(anomaly_Z[:, l8_row], inv_C[l8_row][:,l8_row], sum(l8_row))
result.shape

(8, 625)

In [100]:
result

array([[0.40061301, 0.74747792, 1.13981198, ..., 0.28465167, 1.24688593,
        0.68361752],
       [0.43803768, 2.40551361, 2.75134722, ..., 0.81295017, 2.43919113,
        1.9077597 ],
       [0.57555686, 1.419828  , 0.60372719, ..., 0.25714209, 0.27681278,
        0.63163231],
       ...,
       [0.95844302, 1.62840704, 0.84582724, ..., 0.11961682, 0.70393199,
        0.98730226],
       [0.02832992, 0.42913806, 2.19415484, ..., 0.31482615, 1.99200285,
        1.37579142],
       [0.87907797, 2.01629125, 2.80855207, ..., 0.35973117, 2.24699251,
        1.66191773]])

In [101]:
#SN比
sn = np.zeros(l8.shape[0])
for idx, row in enumerate(result):
    sum_MD = 0
    for i in range(len(row)):
        sum_MD += 1 / row[i]
    sn[idx] = -10 * math.log10(sum_MD / len(row))
sn

array([-1.73378917,  0.45191467, -5.25741369, -4.97371304, -6.94612566,
       -5.8971606 , -2.59689125,  0.28587321])

In [110]:
#変数選択
df_sn = pd.DataFrame(index=X_train.columns, columns=['SN比','残す'])
for i, clm in enumerate(X_train.columns):
    df_sn.loc[df_sn.index == clm, 'SN比'] = sum(sn[l8.T[i]]) - sum(sn[~l8.T[i]])
    df_sn.loc[df_sn.index == clm, '残す'] = sum(sn[l8.T[i]]) - sum(sn[~l8.T[i]]) > 0
df_sn
# select_v = df_sn.reset_index()

,SN比,残す
5,3.641303,True
10,-1.583016,False
14,19.48152,True
13,-6.401134,False
2,1.462325,True
16,-0.068204,False
12,-3.735803,False


In [103]:
normal_X_d = normal_X.drop(columns=df_sn[df_sn['残す'] == False].index)
anomaly_X_d = anomaly_X.drop(columns=df_sn[df_sn['残す'] == False].index)

normal_Z_d = transform_standard(normal_X_d, normal_X_d)
anomaly_Z_d = transform_standard(normal_X_d, anomaly_X_d)

#共分散行列の逆行列
inv_C_d = inv_cov(normal_Z_d)

#マハラノビス距離の計算
normal_MD = cal_MD(normal_Z_d, inv_C_d, len(normal_X_d.columns))
anomaly_MD = cal_MD(anomaly_Z_d, inv_C_d, len(normal_X_d.columns))

In [104]:
Z = transform_standard(normal_X_d, X_test.drop(columns=df_sn[df_sn['残す'] == False].index))
inv_C_d = inv_cov(normal_Z_d)

MD = cal_MD(Z, inv_C_d, len(normal_X_d.columns))
MD

[2.1078370485273523,
 2.3205943948502394,
 2.34447046587194,
 0.5719331550538526,
 0.9176586897716038,
 1.9930523682515104,
 1.305602128842925,
 0.6113072451605722,
 2.163735849734715,
 0.7242177008767814,
 1.7495249973925329,
 1.0223847407986786,
 0.6415783759972447,
 0.903245846183883,
 0.32601009670593056,
 0.9198830868426944,
 1.5932712134612508,
 0.0569973706244768,
 1.4925606763713288,
 0.6306056447289281,
 0.3327893181587824,
 0.6306056447289281,
 0.897635141625588,
 0.061573212027555606,
 0.5254348061836517,
 1.1504391567752652,
 0.4731543523538074,
 0.061573212027555606,
 1.0010123474556702,
 0.4002027306215243,
 0.6645493600504386,
 0.8826303980907966,
 0.3327893181587824,
 1.6268346410428167,
 0.052188091359477606,
 2.194989172534473,
 0.27033062791476753,
 3.511226041552962,
 0.6530138219632317,
 1.0536129614677008,
 2.4957232012369066,
 0.3247380375649719,
 1.5650071921882194,
 0.897635141625588,
 0.5483804746913051,
 0.061573212027555606,
 1.878996709017853,
 0.8857735631

In [105]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test.values, MD) 

0.6544545893842673

In [106]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [107]:
clf = LogisticRegression(solver="liblinear", random_state=0).fit(use_X, y)
roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

0.8810180828607035

In [108]:
confusion_matrix(y_test, clf.predict(X_test))

array([[3827,    9],
       [ 158,    6]], dtype=int64)